# Étude de la pollution de l'air en Occitanie

La pollution de l'air est principalement dûes aux activités humaines, et impactent notre environnement et notre santé.   
Sur ce site, vous trouverez une étude de l'évolution de la pollution de l'air en Occitanie.
Cette étude permet une visulalisation de l'évolution des concentrations de polluants en fonction du temps.


## Qualité de l'air en Occitanie


In [ ]:
#| echo: false

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import requests
import warnings 
warnings.filterwarnings("ignore")
# initialisation d'un dataframe à partir d'un fichier local contenant les géométries (polygone) de l'Occitanie
couche = gpd.read_file("./modpollution/data/polyg_geom.geojson")

# affichage des 5 premières lignes afin d'avoir un aperçu des données
couche.head()

# initialisation d'un nouveau dataframe à partir d'un URL (ATMO) pour les données de pollutions
geopollu = gpd.read_file("https://services9.arcgis.com/7Sr9Ek9c1QTKmbwr/arcgis/rest/services/mesures_occitanie_72h_poll_princ/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")


# affichage du dataframe
geopollu.head()

# ici on reassigne la colone geometry du datafrafe couche au datataframe geopollu et on le renomme polyg_geom
geopollu = geopollu.assign(polyg_geom=couche['geometry'])
geopollu = geopollu.assign(nom_officiel_departement=couche['nom_officiel_departement'])
geopollu.head()

geopollu.drop('geometry', axis=1, inplace=True)#(cette ligne permet de supprimer la colone geometry du dataframe geopollu)
geopollu.drop('nom_dept', axis=1, inplace=True)

# Assurez-vous que la colonne géométrique est correctement définie
geopollu = geopollu.set_geometry('polyg_geom')

# Supprimez les lignes avec des géométries nulles
geopollu = geopollu.dropna(subset=['polyg_geom'])

# Création d'une figure et des axes
fig, ax = plt.subplots(figsize=(7, 7))

# On trace le GeoDataFrame
geopollu.plot(ax=ax, color='lightgrey', edgecolor='green')

# On ajoute des cercles proportionnels à la quantité de polluant
for idx, row in geopollu.iterrows():
    if row['polyg_geom']:
        ax.plot(row.polyg_geom.centroid.x, row.polyg_geom.centroid.y, 'ro', markersize=row['valeur'] * 0.1)
        # Ajouter le nom du département
        ax.text(row.polyg_geom.centroid.x, row.polyg_geom.centroid.y, row['nom_officiel_departement'], fontsize=8)


# Ajouter le légende
ax.set_title('Concentration moyenne de tous les polluants')
plt.show()

# Enregistrer le plot en tant que fichier SVG
fig.savefig('La_quantité_de_polluant.svg', format='svg')

Cette première carte permet de voir la concentration de polluants de l'air sur l'année 2023. Plus le point rouge est gros, plus la concentration est élevée. On remarque que le département de l'Hérault a été le plus touché par la pollution de l'air lors de cette dernière année. On pourra retrouver plus d'information à ce sujet dans la page dédiée à ce département.  


In [ ]:
#| echo: false

import requests
import folium
from datetime import datetime
from IPython.display import display
import os
import warnings 
warnings.filterwarnings("ignore")

# Récupérer les données depuis l'URL
url = "https://services9.arcgis.com/7Sr9Ek9c1QTKmbwr/arcgis/rest/services/epipol_occitanie/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"
response = requests.get(url)
data = response.json()

# Créer une carte centrée sur une position approximative en Occitanie
m = folium.Map(location=[43.6, 1.4], zoom_start=8)

# Créer un dictionnaire pour stocker le nombre d'alertes et les dates associées à chaque point
alerts_by_location = {}

# Ajouter des marqueurs rouges pour chaque alerte avec le nombre d'alertes, la date et le polluant responsable
features = data.get('features', [])
for feature in features:
    properties = feature.get('properties', {})
    geometry = feature.get('geometry', {})
    coordinates = geometry.get('coordinates', [])
    etat = properties.get('etat', '')
    date_ech = properties.get('date_ech', '')
    lib_zone = properties.get('lib_zone', '')
    lib_pol = properties.get('lib_pol', '')  # Ajout du polluant responsable

    # Vérifier si l'état est une alerte et si la date est présente
    if etat == 'ALERTE' and date_ech:
        date_str = datetime.fromtimestamp(date_ech / 1000).strftime('%m/%d/%Y')

        # Ajouter le nombre d'alertes, la date et le polluant responsable au dictionnaire associé aux coordonnées
        if tuple(coordinates) in alerts_by_location:
            alerts_by_location[tuple(coordinates)]['count'] += 1
            alerts_by_location[tuple(coordinates)]['dates'].append(f"{date_str} ({lib_pol})")
        else:
            alerts_by_location[tuple(coordinates)] = {'count': 1, 'dates': [f"{date_str} ({lib_pol})"], 'lib_zone': lib_zone}

# Ajouter des marqueurs pour chaque point avec une taille de marqueur variable en fonction du nombre d'alertes
for coordinates, alert_info in alerts_by_location.items():
    count = alert_info['count']
    lib_zone = alert_info['lib_zone']
    dates = "<br>".join(alert_info['dates'])

    # La taille du marqueur est définie par le nombre d'alertes
    radius = count * 2  # Vous pouvez ajuster le facteur de multiplication pour obtenir la taille désirée

    popup_text = f"Nombre d'alertes : {count}<br>Dates : {dates}<br>Département : {lib_zone}"
    
    # Créer le marqueur avec une taille variable
    folium.CircleMarker(location=[coordinates[1], coordinates[0]], radius=radius, popup=popup_text, color='red', fill=True).add_to(m)

# Afficher la carte dans le notebook
display(m)

Cette carte interractive permet de voir tout les pics de pollution de l'air au cours de la dernière année. Ces pics sont répertoriés sous forme d'Alertes, dont on connaît la date. La plupart de ces alertes concernent les particules PM10 (surtout durant les mois d'hiver), ainsi que l'O3 (plutôt en été).   
   

Cela n'a rien d'étonnant quand on comprend l'origine et l'impact des polluants. Ces pourquoi on propose un court descriptifs des polluants étudiés.

## Description des différents polluants

### PM 10
Les particules PM10 sont des particules fines dont le diamètre inférieur à 10 µm. Elles peuvent provenir de différentes sources comme les chantiers, les carrières ou encore l'industrie manufacturière. Ces particules provoquent une altération de la santé respiratoire.

### PM2.5
Les particules PM2.5 sont des particules fines dont le diamètre est inférieur à 2.5 µm. Elles entraîne de plus une altération de la santé cardiovasculaire, et proviennent princiaplement du secteur résidentiel et du transport routier.

### NOX
Les NOX regroupent différents oxydes d'azote : monoxyde d'azote, dioxyde d'azote et protoxyde d'azote. Ces composés chimiques composés d'oxygène et d'azote sont des polluants atmosphériques qui contribuent fortement à l'effet de serre. Ce sont des indicateurs des activités de combustion, notamment le trafic routier.

#### NO2
Le dioxyde d’azote (NO2) a pour source principale le trafic routier. Il entraine une diminution de la fonction pulmonaire, et, à des concentrations dépassant 200 μg/m3, sur de courtes durées, c'est un gaz toxique entraînant une inflammation importante des voies respiratoires.
Ce gaz participe au phénomène des pluies acides, qui appauvrissent les milieux naturels.

#### NO

### 

### 

On se propose donc de visualiser les principaux polluants de notre étude. 

## Visualisation des concentrations des principaux polluants en Occitanie en 2023


In [ ]:
#| echo: false

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import requests
import warnings 
warnings.filterwarnings("ignore")

# initialisation d'un dataframe à partir d'un fichier local contenant les géométries (polygone) de l'Occitanie
couche = gpd.read_file("./modpollution/data/polyg_geom.geojson")

# Définir l'URL
url = "https://services9.arcgis.com/7Sr9Ek9c1QTKmbwr/arcgis/rest/services/Indice_quotidien_de_qualité_de_l’air_pour_les_collectivités_territoriales_en_Occitanie/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"
# url = "https://services9.arcgis.com/7Sr9Ek9c1QTKmbwr/arcgis/rest/services/mesures_occitanie_72h_poll_princ/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

# Effectuer une requête pour récupérer les données GeoJSON
response = requests.get(url)
data = response.json()

# Créer un GeoDataFrame à partir des données GeoJSON
gdf = gpd.GeoDataFrame.from_features(data['features'])

# gdf = gdf.merge(couche[['geometry']], left_index=True, right_index=True, suffixes=('', '_couche'))
# geopollu = gdf.rename(columns={'geometry_couche': 'polyg_geom'})

# On assigne les colones 'geometry' et 'nom_officiel_departement' au geodataframe gdf
gdf = gdf.assign(geometry=couche['geometry'])
gdf = gdf.assign(nom_officiel_departement=couche['nom_officiel_departement'])

# Afficher les premières lignes du GeoDataFrame
gdf.head()


# Créer une figure avec deux sous-plots côte à côte
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot 1 : Carte avec la colonne 'code_o3'
axs[0].set_title('Concentration du polluant O3')
gdf.plot(column='code_o3', cmap='Spectral', linewidth=0.8, ax=axs[0], edgecolor='0.8', legend=True)

# Ajouter des étiquettes pour chaque département
for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf['nom_officiel_departement']):
    axs[0].annotate(label, xy=(x, y), xytext=(3, 3), textcoords='offset points')

# Plot 2 : Carte avec la colonne 'code_pm10'
axs[1].set_title('Concentration du polluant PM10')
gdf.plot(column='code_pm10', cmap='Spectral', linewidth=0.8, ax=axs[1], edgecolor='0.8', legend=True)

# Ajouter des étiquettes pour chaque département
for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf['nom_officiel_departement']):
    axs[1].annotate(label, xy=(x, y), xytext=(3, 3), textcoords='offset points')

# Ajuster l'espacement entre les sous-plots
plt.tight_layout()

# Afficher les plots
plt.show()


# Créer une figure avec deux sous-plots côte à côte
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot 1 : Carte avec la colonne 'code_no2'
axs[0].set_title('Concentration du polluant NO2')
gdf.plot(column='code_no2', cmap='Spectral', linewidth=0.8, ax=axs[0], edgecolor='0.8', legend=True)

# Ajouter des étiquettes pour chaque département
for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf['nom_officiel_departement']):
    axs[0].annotate(label, xy=(x, y), xytext=(3, 3), textcoords='offset points')

# Plot 2 : Carte avec la colonne 'code_pm25'
axs[1].set_title('Concentration du polluant PM2.5')
gdf.plot(column='code_pm25', cmap='Spectral', linewidth=0.8, ax=axs[1], edgecolor='0.8', legend=True)

# Ajouter des étiquettes pour chaque département
for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf['nom_officiel_departement']):
    axs[1].annotate(label, xy=(x, y), xytext=(3, 3), textcoords='offset points')

# Ajuster l'espacement entre les sous-plots
plt.tight_layout()

# Afficher les plots
plt.show()


gdf = gdf.set_geometry('geometry')
# Création d'une figure et des axes
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

# On trace les départements avec différentes couleurs en fonction de la quantité de polluant
ax.set_title('Concentration du polluant SO2')
gdf.plot(column='code_so2', cmap='Spectral', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
# gdf.plot(column='code_pm10', cmap='Spectral', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

# On ajouter des étiquettes pour chaque département
for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf['nom_officiel_departement']):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords='offset points')

## Évolution des concentrations des polluants au fil du temps 

### Évolution des concentrations des polluants au cours des 5 dernières années


In [ ]:
#| echo: false

import modpollution
import warnings 
warnings.filterwarnings("ignore")
df_a = modpollution.as_df("./modpollution/data/data_a.json")
df_a = modpollution.modif_date2(df_a)
df_a = modpollution.mean_df(df_a)

modpollution.plotpollline(df_a,color='nom_poll',titre='Moyenne des concentrations par an en Occitanie')

Le graphique ci-dessus représente l'évolution de la concentration des différents polluant par années en Occitanie.   
Ils permettent une première visualisation rapide des données annuelles.
On remarque que les concentrations des polluants sont plutôt stables, avec une petite chute en 2020, lors de la pandémie du Covid-19.   
Ce graphique est interractif! Le curseur vous permet de selectionner une zone particulière et de naviguer dans le temps.  


In [ ]:
#| echo: false

import modpollution
import warnings 
warnings.filterwarnings("ignore")
df_a = modpollution.as_df("./modpollution/data/data_a.json")
df_a = modpollution.modif_date2(df_a)
df_a = modpollution.mean_df(df_a)

modpollution.plotpoll(df_a)

Le second graphique représente exactement les mêmes données, mais de manière annimée. Vous pouvez utiliser la barre de temps pour vous déplacer ou simplement laisser l'animation se faire.


In [ ]:
#| echo: false
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from collections import defaultdict
import warnings 
warnings.filterwarnings("ignore")

# URL of the GeoJSON API for the entire Occitanie region
url = "https://services9.arcgis.com/7Sr9Ek9c1QTKmbwr/arcgis/rest/services/mesures_occitanie_annuelle_poll_princ/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

# Get GeoJSON data from the API
response = requests.get(url)
data = response.json()

# Extract relevant data for the Occitanie region
features = data['features']
occitanie_data = [feature['properties'] for feature in features]

# Prepare data for plotting
pollutants_per_year = defaultdict(list)

for entry in occitanie_data:
    pollutant = entry['nom_poll']
    concentration = entry['valeur']
    date_debut_timestamp = entry['date_debut']
    year = datetime.utcfromtimestamp(date_debut_timestamp / 1000).year

    pollutants_per_year[pollutant].append((year, concentration))

# Transform data for plotting (calculate average concentration per year)
for pollutant, data in pollutants_per_year.items():
    data.sort(key=lambda x: x[0])

    # Calculate average concentrations per year
    averages_per_year = defaultdict(float)
    counts_per_year = defaultdict(int)

    for year, concentration in data:
        if concentration is not None:
            averages_per_year[year] += concentration
            counts_per_year[year] += 1

    for year in averages_per_year:
        if counts_per_year[year] > 0:
            averages_per_year[year] /= counts_per_year[year]

    pollutants_per_year[pollutant] = list(averages_per_year.items())

# Plotting
for pollutant, data in pollutants_per_year.items():
    years, concentrations = zip(*data)
    plt.plot(years, concentrations, marker='o', linestyle='-', linewidth=1, label=pollutant)

plt.title('Concentration des polluants en Occitanie par année')
plt.xlabel('Année')
plt.ylabel('Concentration')
plt.legend(title='Pollutant', loc='upper right')
plt.show()